In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Not connected to a GPU


In [2]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import asarray
from numpy import savetxt
import pandas as pd
from numpy import loadtxt
import os, datetime

In [3]:
# NN-Tools
# Shortcuts to keras if (however from tensorflow)
import tensorflow as tf
from tensorflow import keras
#from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

In [68]:
url_wind_dir = 'https://raw.githubusercontent.com/regiiis/nn_windfarm/main/data/wind_dir.csv'
url_farm = 'https://raw.githubusercontent.com/regiiis/nn_windfarm/main/data/windfarm.csv'

wind_dir = pd.read_csv(url_wind_dir)
wind_v = pd.read_csv(url_farm)

# merge two dataframes
df = wind_dir.merge(wind_v, left_on='Unnamed: 0', right_on='Unnamed: 0')


In [81]:
# Wind Direction Analysis

# Drop columns with inconsistent wind_dir records
# Get mean wind direction per angle, between average of 80.0 and 100.0. Group wind_dir by steps of 10.
# Rename numerical titles
df_main_2 = df.drop(columns = ['10.0'])
df_main_2 = df_main_2.drop(columns = ['40.0'])
df_main_2 = df_main_2.drop(columns = ['60.0'])
df_main_2 = df_main_2.drop(columns = ['Unnamed: 0'])
#df1['wind_dir'] = ((df1['80.0'] + df1['100.0'])/2).round(0)
df_main_2['80.0'] = df_main_2['80.0'].round(0)
df_main_2['100.0'] = df_main_2['100.0'].round(0)
df_main = df_main_2.rename(columns={'80.0': 'eighty', '100.0': 'hundred'}).copy()

df_main.head(4)

,eighty,hundred,WTG1,WTG2,WTG3,WTG4,WTG5,WTG6,WTG7,WTG8,...,WTG23,WTG24,WTG25,WTG26,WTG27,WTG28,WTG29,WTG30,WTG31,WTG32
0,111.0,113.0,11.3,12.4,10.4,9.8,9.9,10.5,11.4,10.5,...,13.4,12.5,12.2,12.6,12.5,12.8,13.8,12.9,12.4,8.9
1,113.0,116.0,11.6,11.9,10.4,10.7,10.6,10.7,12.0,9.8,...,12.7,13.0,12.5,14.4,12.4,12.8,13.4,13.1,13.2,9.8
2,110.0,112.0,11.8,12.5,10.8,10.6,10.4,10.5,11.9,10.0,...,12.2,12.1,11.7,12.2,12.0,12.2,13.2,12.4,12.3,8.7
3,113.0,116.0,11.7,11.6,10.6,10.4,10.1,10.3,11.7,9.2,...,12.8,13.2,10.9,12.4,12.3,12.3,12.4,11.8,10.5,8.6


In [77]:
df_main = tf.convert_to_tensor(df_main)
df_main[1]

<tf.Tensor: shape=(34,), dtype=float64, numpy=
array([113. , 116. ,  11.6,  11.9,  10.4,  10.7,  10.6,  10.7,  12. ,
         9.8,  11.8,  11.6,  11. ,   9.9,  10.5,  12. ,  12. ,  12. ,
        12.3,  12.1,  12.1,  12.1,  10.1,  12.6,  12.7,  13. ,  12.5,
        14.4,  12.4,  12.8,  13.4,  13.1,  13.2,   9.8])>

In [88]:
# Create train, validate and test data set.
reduced_df = df_main.sample(frac=0.8,random_state=200) # random state is a seed value

train_df_2 = reduced_df.sample(frac=0.8,random_state=200) # random state is a seed value
train_df = tf.convert_to_tensor(train_df_2)

validation_test_df_2 = reduced_df.drop(train_df_2.index)
validation_test_df = tf.convert_to_tensor(reduced_df.drop(train_df_2.index))

validation_df_2 = validation_test_df_2.sample(frac=0.5,random_state=200)
validation_df = tf.convert_to_tensor(validation_test_df_2.sample(frac=0.5,random_state=200))

test_df = tf.convert_to_tensor(validation_test_df_2.drop(validation_df_2.index))


In [89]:
# Build and compile model with Keras
model = keras.Sequential()
model.add(Dense(34, input_shape=(34,), activation='relu'))
model.add(Dense(34, activation='relu'))
model.add(Dense(34, activation='relu'))
model.add(Dense(34, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_65 (Dense)            (None, 34)                1190      
                                                                 
 dense_66 (Dense)            (None, 34)                1190      
                                                                 
 dense_67 (Dense)            (None, 34)                1190      
                                                                 
 dense_68 (Dense)            (None, 34)                1190      
                                                                 
Total params: 4,760
Trainable params: 4,760
Non-trainable params: 0
_________________________________________________________________


In [90]:
# Fit the Keras model on the dataset

logdir = os.path.join("logs_feature_extraction", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="feature_extraction.h5", save_best_only=True, monitor="val_loss"),
    tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
]

history = model.fit(
    train_df,
    epochs = 60,
    batch_size=10,
    validation_data = validation_df,
    callbacks = callbacks
)

ValueError: ignored